In [31]:
import random
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Dense, LSTM, Concatenate, Input, Conv1D, Flatten, Dropout, BatchNormalization, \
    GlobalAveragePooling1D, MaxPooling1D, GlobalAveragePooling1D, UpSampling1D, concatenate, Conv1DTranspose, \
    Cropping1D, UpSampling1D, ZeroPadding1D, Reshape, Lambda, Multiply, Softmax, Add
from keras.models import Sequential, Model

matplotlib.use('TkAgg')

In [7]:
data = pd.read_csv("data/DataBase_RF_bi_lowperc_1.csv")

In [8]:
data

,name,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2,S1 #1,S1' #1,S2 #1,S3 #1,S1 #2,S1' #2,S2 #2,S3 #2
0,Structure: Al_80_Si_112_Boulfelfel_Ca_1_K_78,80,112,1,78,1.00,1.38,22.80,43.40,2,1,0,0,1,0,16,7,34,21
1,Structure: Al_80_Si_112_Boulfelfel_Ca_1_Na_78,80,112,1,78,1.00,1.02,22.80,24.11,2,1,0,0,1,0,1,25,30,22
2,Structure: Al_80_Si_112_Boulfelfel_Ca_1_Rb_78,80,112,1,78,1.00,1.52,22.80,47.30,2,1,1,0,0,0,14,8,35,21
3,Structure: Al_80_Si_112_Boulfelfel_Cs_1_K_79,80,112,1,79,1.67,1.38,59.42,43.40,1,1,0,0,1,0,16,8,35,20
4,Structure: Al_80_Si_112_Boulfelfel_Cs_1_Li_79,80,112,1,79,1.67,0.76,59.42,24.33,1,1,0,0,0,1,0,23,31,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Structure: Al_96_Si_96_Boulfelfel_Rb_14_Na_82,96,96,14,82,1.52,1.02,47.30,24.11,1,1,0,0,0,14,0,32,32,18
496,Structure: Al_96_Si_96_Boulfelfel_Rb_15_Na_81,96,96,15,81,1.52,1.02,47.30,24.11,1,1,0,0,1,14,0,32,31,18
497,Structure: Al_96_Si_96_Boulfelfel_Rb_17_K_79,96,96,17,79,1.52,1.38,47.30,43.40,1,1,2,2,4,9,14,5,35,25
498,Structure: Al_96_Si_96_Boulfelfel_Rb_4_Li_92,96,96,4,92,1.52,0.76,47.30,24.33,1,1,0,0,1,3,0,32,31,29


In [9]:
data.iloc[22]

name                 Structure: Al_81_Si_111_Boulfelfel_Cs_2_Li_79
Al                                                              81
Si                                                             111
Cation #1                                                        2
Cation #2                                                       79
Radius #1                                                     1.67
Radius #2                                                     0.76
Polarizability #1                                            59.42
Polarizability #2                                            24.33
Charge #1                                                        1
Charge #2                                                        1
S1 #1                                                            0
S1' #1                                                           0
S2 #1                                                            0
S3 #1                                                         

In [24]:
data.loc[(data["Cation #1"] != (data["S1 #1"] + data["S1\' #1"] + data["S2 #1"] + data["S3 #1"]))
         |
         (data["Cation #2"] != (data["S1 #2"] + data["S1\' #2"] + data["S2 #2"] + data["S3 #2"]))][["Cation #1", "S1 #1", "S1\' #1", "S2 #1", "S3 #1"]]

,Cation #1,S1 #1,S1' #1,S2 #1,S3 #1
22,2,0,0,0,1
30,1,0,1,0,0
47,3,0,0,1,1
68,2,0,1,0,0
70,3,0,0,1,1
106,5,0,1,1,2
138,4,4,0,0,0
160,1,0,0,0,0
194,6,0,0,3,2
225,9,0,2,3,3


In [25]:
data_without_names = data.drop(columns=['name'])
plt.matshow(data_without_names.corr())
continuous_features = data_without_names.describe().columns
plt.xticks(range(len(continuous_features)), continuous_features)
plt.yticks(range(len(continuous_features)), continuous_features)
plt.colorbar()
plt.show()

In [27]:
features = data.drop(columns=["name", "S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"],axis=1)
labels = data[["S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"]]

In [160]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.25,
                                                                            random_state=0)

In [29]:
features_train

,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2
239,88,104,1,87,1.02,1.67,24.11,59.42,1,1
59,82,110,80,1,1.02,1.00,24.11,22.80,1,2
406,94,98,7,80,1.00,1.52,22.80,47.30,2,1
391,93,99,2,91,1.02,1.38,24.11,43.40,1,1
411,94,98,92,1,1.67,1.00,59.42,22.80,1,2
...,...,...,...,...,...,...,...,...,...,...
323,91,101,6,85,1.02,1.38,24.11,43.40,1,1
192,87,105,2,85,1.67,1.52,59.42,47.30,1,1
117,84,108,78,3,0.76,1.00,24.33,22.80,1,2
47,82,110,3,79,1.67,1.38,59.42,43.40,1,1


In [222]:
from sklearn.preprocessing import MinMaxScaler

# scale_features_mm = MinMaxScaler()
# features_train = scale_features_mm.fit_transform(features_train)
# features_test = scale_features_mm.transform(features_test)

scale_labels_mm = MinMaxScaler()
labels_train = scale_labels_mm.fit_transform(labels_train)
labels_test = scale_labels_mm.transform(labels_test)

In [383]:
features_train

,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2
454,95,97,9,86,0.76,1.38,24.33,43.40,1,1
416,94,98,2,92,0.76,1.52,24.33,47.30,1,1
167,86,106,3,83,1.38,1.52,43.40,47.30,1,1
165,86,106,3,83,1.38,1.67,43.40,59.42,1,1
52,82,110,3,79,1.38,1.52,43.40,47.30,1,1
...,...,...,...,...,...,...,...,...,...,...
335,92,100,12,80,1.67,0.76,59.42,24.33,1,1
201,87,105,2,85,0.76,1.38,24.33,43.40,1,1
123,84,108,5,79,1.02,1.67,24.11,59.42,1,1
50,82,110,3,79,1.38,0.76,43.40,24.33,1,1


In [176]:
def mps_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    delta = tf.abs(y_true - y_pred)
    delta_ = tf.reduce_mean(delta)
    sigma = tf.sqrt(tf.reduce_mean(tf.square(delta - delta_)))
    size = tf.size(delta, out_type=tf.int32)
    return tf.reduce_mean(delta_ / tf.cast(size, tf.float32) * (sigma / tf.cast(size, tf.float32) + 1))

def set_model():
    #shared
    inputs = Input(shape=(3,), name="struct_input")
    x = Dense(3, activation='relu', name="struct_d1")(inputs)
    x = Dense(3, activation='relu', name="struct_d2")(x)
    feature_ext_model = Model(inputs, x)
    #
    struct1, struct2 = Input(shape=(3,)), Input(shape=(3,))
    out1, out2 = feature_ext_model(struct1), feature_ext_model(struct2)
    y = Add()([out1, out2])
    y = Dense(8, activation='relu', name="d1")(y)
    model = Model([out1, out2], y)
    model.compile(loss="mse", optimizer='Adam')
    return model



def create_model(x):
    inputs = Input(shape=10)
    num_elements = Input(shape=1)
    x = Dense(10, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    d = Dense(8, activation='relu')(x)
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(d)
    outputs = Multiply()([l1, num_elements])
    model = tf.keras.Model(inputs=[inputs, num_elements], outputs=outputs)
    model.compile(loss=mps_loss, optimizer='Adam')
    return model


model = set_model()#create_model(features_train)

In [177]:
def generate_data_for_set():
    f_train = [[], []]
    f_test = [[], []]
    for i in range(len(features_train)):
        f_train[0].append(features_train[["Cation #1", "Radius #1", "Polarizability #1"]].iloc[i])
        f_train[1].append(features_train[["Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
    for i in range(len(features_test)):
        f_test[0].append(features_test[["Cation #1", "Radius #1", "Polarizability #1"]].iloc[i])
        f_test[1].append(features_test[["Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
        
    f_train[0] = np.array(f_train[0])
    f_train[1] = np.array(f_train[1])
    f_test[0] = np.array(f_test[0])
    f_test[1] = np.array(f_test[1])
    return f_train, f_test

f_train, f_test = generate_data_for_set()

# num_train = []
# num_test = []
# for i in range(len(features_train)):
#     #labels_train.iloc[i] /= np.sum(labels_train.iloc[i])
#     num_train.append(features_train.iloc[i]["Al"])
# for i in range(len(features_test)):
#     #labels_test.iloc[i] /= np.sum(labels_test.iloc[i])
#     num_test.append(features_test.iloc[i]["Al"])

In [179]:
def plot():
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plot()


mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/mps.keras', save_best_only=True,
                                              monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
                                              save_freq='epoch')

# history = model.fit([features_train, np.array(num_train)], labels_train,
#                     epochs=100, batch_size=16, shuffle=True,
#                     validation_data=([features_test, np.array(num_test)], labels_test), callbacks=[mcp_save])

history = model.fit((f_train[0], f_train[1]), labels_train,
                    epochs=100, batch_size=16, shuffle=True,
                    validation_data=((f_test[0], f_test[1]), labels_test), callbacks=[mcp_save])


plot_loss(history)

Epoch 1/100
 1/24 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 31.7990
Epoch 1: val_loss improved from inf to 77.70302, saving model to data/models/mps.keras
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 49.4639 - val_loss: 77.7030
Epoch 2/100
 1/24 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 70.9625
Epoch 2: val_loss improved from 77.70302 to 77.52870, saving model to data/models/mps.keras
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 58.4496 - val_loss: 77.5287
Epoch 3/100
 1/24 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 53.8950
Epoch 3: val_loss improved from 77.52870 to 77.36109, saving model to data/models/mps.keras
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 51.6267 - val_loss: 77.3611
Epoch 4/100
 1/24 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 53.4379
Epoch 4: val_loss improved from 77.36109 to 76.90681, saving model to data/models/mps.keras
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 52.2625 - val_loss: 76.9068
Epoch 5/100
 1/24 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 71.05

In [180]:
model = tf.keras.models.load_model('data/models/mps.keras')

ValueError: Layer node index out of bounds.
inbound_layer = <InputLayer name=keras_tensor_142CLONE, built=True>
inbound_layer._inbound_nodes = [<Node operation=keras_tensor_142CLONE, id=2276974555440>]
inbound_node_index = 1

In [181]:
#predictions = model.predict([features_test, np.array(num_test)])
predictions = model.predict((f_test[0], f_test[1]))
predictions_2 = model.predict((f_test[1], f_test[0]))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

C:\Users\anton\PycharmProjects\isotherm-analyzer\venv\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_141CLONE', 'keras_tensor_142CLONE']. Received: the structure of inputs=('*', '*')
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [182]:
features_test.iloc[88]

Al                    92.00
Si                   100.00
Cation #1             88.00
Cation #2              2.00
Radius #1              1.38
Radius #2              1.00
Polarizability #1     43.40
Polarizability #2     22.80
Charge #1              1.00
Charge #2              2.00
Name: 348, dtype: float64

In [187]:
import random


def mps(first, second):
    delta = []
    for i in range(len(first)):
        delta.append(abs(first[i] - second[i]))
    delta_ = sum(delta) / len(delta)
    sigma = 0
    for i in range(len(first)):
        sigma += (delta_ - abs(first[i] - second[i])) ** 2
    sigma = np.sqrt(sigma / len(delta))
    return delta_ / len(delta) * (sigma / len(delta) + 1)


i = random.randint(0, len(predictions) - 1)
print("№", i)
rounded_prediction = [round(p, 1) for p in predictions[i]]
print(rounded_prediction)
rounded_prediction2 = [round(p, 1) for p in predictions_2[i]]
print(rounded_prediction2)


rounded_real = [round(float(p), 1) for p in np.array(labels_test.iloc[i])]
print(rounded_real)
print(sum(predictions[i]), sum(labels_test.iloc[i]))

№ 27
[1.5, 0.0, 4.9, 4.6, 7.1, 15.2, 30.8, 25.3]
[1.5, 0.0, 4.9, 4.6, 7.1, 15.2, 30.8, 25.3]
[0.0, 0.0, 0.0, 9.0, 1.0, 30.0, 32.0, 20.0]
89.36898684501648 92


In [188]:
mps_list = []
for i in range(len(predictions)):
    p = predictions[i]
    real = np.array(labels_test.iloc[i])
    mps_list.append(mps(p, real))
print(sum(mps_list))

141.27128075635667


In [716]:
plt.hist(mps_list)
plt.show()

In [610]:
print(sum(mps_list))

37.59565462806723
